# Medição da espessura de um fio de cabelo por difração

$$d = \frac{2\cdot m\cdot\lambda\sqrt{x^2 + (y_m/2)^2}}{y_m}$$
$$y_m = \frac{Y_{\text{max}}+Y_{\text{min}}}{2}$$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.medicoes import Me, average, sqrt, list2numpy

def espessura(lambda_:Me, x:Me, y_m:Me, m:int) -> Me:
    return 2*m*lambda_*sqrt(x**2+(y_m/2)**2)/y_m

def espessura_arr(lambda_:Me, x:Me, y:list[Me]):
    arr:list[Me] = []
    for m, y_m in enumerate(y):
        arr.append(espessura(lambda_, x, y_m, m+1))
    return arr

## Dados

In [2]:
lambda_ = Me(670, 10, 10**-9) # [Hz] - Frequência do laser 

def trena(m: float): # [cm] - Trena
    return Me(m, 0.1, 10**-2)
def regua(m: float): # [cm] - Regua
    return Me(m, 0.05, 10**-2)
def paqui(m: float): # [mm] - Paquimetro
    return Me(m, 0.05, 10**-3)
def micro(m: float): # [mm] - Micrometro
    return Me(m, 0.005, 10**-3)

pessoas = ["lucas", "pamela", "gabi"]
dados = {
    pessoas[0]: {
        "espessura_micrometro": None,
        "x": None,
        "y_arr": None
    },
    pessoas[1]: {
        "espessura_micrometro": None,
        "x": None,
        "y_arr": None
    },
    pessoas[2]: {
        "espessura_micrometro": None,
        "x": None,
        "y_arr": None
    },
}

### Cabelo Lucas

In [17]:
pessoa = pessoas[0]
dados[pessoa]["espessura_micrometro"] = micro(0.071)
dados[pessoa]["x"] = trena(267.75)-trena(30) # [m] - Distância do fio de cabelo com o anteparo
dados[pessoa]["y_arr"] = [   # Distância entre os máximo de ordem m+1
    (paqui(53.55)+paqui(34.45))/2,
    (paqui(84.40)+paqui(64.60))/2,
    (paqui(114.90)+paqui(95.10))/2,
    (paqui(145.00)+paqui(127.20))/2,
    (regua(17.26)+regua(15.53))/2
]

d_arr = espessura_arr(
    lambda_ = lambda_,
    x = dados[pessoa]["x"],
    y = dados[pessoa]["y_arr"]
)
print(dados[pessoa]["espessura_micrometro"])
print(average(d_arr))
dados[pessoa]["y_arr"]

(71.0 ± 5.0)e-6
(88.0 ± 1.3)e-6


[(4400.0 ± 3.5)e-5,
 (7450.0 ± 3.5)e-5,
 (10500.0 ± 3.5)e-5,
 (13610.0 ± 3.5)e-5,
 (1639.5 ± 3.5)e-4]

### Cabelo Pamela

In [4]:
pessoa = pessoas[1]
dados[pessoa]["espessura_micrometro"] = micro(0.049)
dados[pessoa]["x"] = trena(264.7)-trena(30) # [m] - Distância do fio de cabelo com o anteparo
dados[pessoa]["y_arr"] = [   # Distância entre os máximo de ordem m+1
    (paqui(101.20)+paqui(67.15))/Me(2),
    (paqui(155.10)+paqui(123.50))/Me(2),
    (regua(21.35)+regua(18.40))/Me(2)
]

d_arr = espessura_arr(
    lambda_ = lambda_,
    x = dados[pessoa]["x"],
    y = dados[pessoa]["y_arr"]
)
print(dados[pessoa]["espessura_micrometro"])
print(average(d_arr))
d_arr

(49.0 ± 5.0)e-6
(433.5 ± 6.5)e-7


[(373.7 ± 5.6)e-7, (451.7 ± 6.7)e-7, (475.1 ± 7.1)e-7]

### Cabelo Gabi

In [5]:
pessoa = pessoas[2]
dados[pessoa]["espessura_micrometro"] = micro(0.059)
dados[pessoa]["x"] = trena(264.7)-trena(30) # [m] - Distância do fio de cabelo com o anteparo
dados[pessoa]["y_arr"] = [   # Distância entre os máximo de ordem m+1
    (paqui(59.00)+paqui(37.80))/2,
    (paqui(90.75)+paqui(68.95))/2,
    (paqui(122.90)+paqui(102.20))/2,
    (regua(15.8)+regua(13.8))/2,
]

d_arr = espessura_arr(
    lambda_ = lambda_,
    x = dados[pessoa]["x"],
    y = dados[pessoa]["y_arr"]
)
print(dados[pessoa]["espessura_micrometro"])
print(average(d_arr))
d_arr

(59.0 ± 5.0)e-6
(78.2 ± 1.2)e-6


[(649.8 ± 9.7)e-7, (78.8 ± 1.2)e-6, (83.9 ± 1.3)e-6, (85.0 ± 1.3)e-6]

In [7]:
numpy_arr = list2numpy(dados[pessoas[0]]["y_arr"], "list[[u,m]]")
print(dados[pessoas[0]]["y_arr"])
print(numpy_arr)

[(4400.0 ± 3.5)e-5, (7450.0 ± 3.5)e-5, (10500.0 ± 3.5)e-5, (13610.0 ± 3.5)e-5, (1639.5 ± 3.5)e-4]
[[4.40000000e-02 3.53553391e-05]
 [7.45000000e-02 3.53553391e-05]
 [1.05000000e-01 3.53553391e-05]
 [1.36100000e-01 3.53553391e-05]
 [1.63950000e-01 3.53553391e-04]]
